# COMP 5214 and ELEC 5680
## Assignment 1 | Yin, Tianci (20587470)
---
In this problem, we will implement a model to classify digits using the MNIST
dataset. The dataset is available at https://git-disl.github.io/GTDLBench/
datasets/mnist_datasets/ or https://pytorch.org/vision/stable/generated/
torchvision.datasets.MNIST.html. Please use both training and test sets.
You can train your model for 20 epochs. Use a batch size of 64 to train the
model. Please submit a report and code for this assignment. You can use
Pytorch or Tensorflow in this assignment.

In [9]:
# imports
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

In [10]:
# setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [11]:
# predefined hyperparameters
NUM_EPOCHS = 20
BATCH_SIZE = 64

### K-Nearest Neighbours (25 Points)
---
We will implement a K-nearest neighbor model that finds the most similar image
given a test image. We can use the scikit-learn Python library to find the KNN.
We will use the sum of absolute difference (SAD) on all the pixels to measure
the similarity between two images. What will be the accuracy if we find the
nearest neighbor? What will be the accuracy if we find K neighbors? When
we find K, we choose the label that appears most among the KNN images (if
there is a tie, we pick the one with the smallest aggregated SAD). Plot a curve
of accuracy versus K from 1 to 10.

In [12]:
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# predefined
K_CANDIDATES = [i for i in range(1, 11)]  # number of neighbors

def load_mnist_flat(proportion=1.0):
    # setup
    train_set = datasets.MNIST("./_data", train=True, download=True, transform=transforms.ToTensor())
    test_set = datasets.MNIST("./_data", train=False, download=True, transform=transforms.ToTensor())

    # load into numpy arrays with flattened features
    train_X = train_set.data.numpy().reshape(len(train_set), -1)
    train_y = train_set.targets.numpy()
    test_X = test_set.data.numpy().reshape(len(test_set), -1)
    test_y = test_set.targets.numpy()
    
    # proportion
    if proportion < 1.0:
        train_X = train_X[:int(proportion*len(train_X))]
        train_y = train_y[:int(proportion*len(train_y))]
        test_X = test_X[:int(proportion*len(test_X))]
        test_y = test_y[:int(proportion*len(test_y))]

    print("MNIST\ntrain shapes:", train_X.shape, train_y.shape, "\ntest shapes:", test_X.shape, test_y.shape)

    return train_X, train_y, test_X, test_y

After digging through `sklearn.neighbors.KNeighborsClassifier` a little, it seems like the default tie-break behavior is based on training data ordering: 

> From [scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier:~:text=Regarding%20the%20Nearest%20Neighbors%20algorithms%2C%20if%20it%20is%20found%20that%20two%20neighbors%2C%20neighbor%20k%2B1%20and%20k%2C%20have%20identical%20distances%20but%20different%20labels%2C%20the%20results%20will%20depend%20on%20the%20ordering%20of%20the%20training%20data.): 
> _"Regarding the Nearest Neighbors algorithms, if it is found that two neighbors, neighbor k+1 and k, have identical distances but different labels, the results will depend on the ordering of the training data."_

And there doesn't appear to be a way to customize that behavior ([Github Issue](https://github.com/scikit-learn/scikit-learn/issues/21006)).

Though according to:

> From [StackExchange](https://stats.stackexchange.com/questions/144718/how-does-scikit-learn-resolve-ties-in-the-knn-classification): 
> _"Digging a little deeper, the used neigh_ind array is the result of calling the kneighbors method, which (though the documentation doesn't say so) appears to return results in sorted order. So ties should be broken by choosing the class with the point closest to the query point, but this behavior isn't documented and I'm not 100% sure it always happens."_ @Danica

> From [StackExchange](https://stats.stackexchange.com/questions/144718/how-does-scikit-learn-resolve-ties-in-the-knn-classification): 
> _"Seems like the sorting only happens for the brute-force approach, and not the tree-based method?"_ @AllanLRH

we could make use of this behavior, but this still only returns the neighbor with the smallest **individual** SAD, not the smallest **aggregated** SAD.

We give up and implement the classification as well as the tie-break behavior on our own...

In [13]:
def knn_classifier_L1(k, train_X, train_y, test_X):
    # sum of absolute difference (L1 distance / manhattan distance)
    model = NearestNeighbors(n_neighbors=k, metric="manhattan", n_jobs=-1)
    model.fit(train_X)

    # get nearest neighbors
    dist, idx = model.kneighbors(test_X, return_distance=True)
    
    # predict test set
    pred = []

    # custom predict logic
    for i in range(len(test_X)):
        # for each test image, get the most common labels of the nearest neighbors
        neighbor_y = train_y[idx[i]]
        unique_y, count_y = np.unique(neighbor_y, return_counts=True)
        common_y = unique_y[count_y == np.max(count_y)] # shapes: (10000, k)
        
        # tie-break behavior
        if len(common_y) != 1:
            aggregated_sad = {}
            for y in common_y:
                common_y_idx = np.where(neighbor_y == y)[0] # get idx of common_y
                aggregated_sad[y] = np.sum(dist[i][common_y_idx]) # get sum of SAD
            
            pred.append(min(aggregated_sad, key=aggregated_sad.get))
        else:
            pred.append(common_y[0])
    
    return np.array(pred)

We evaluate the accuracy of our classifier using Ks ranging from 1 to 10:

In [ ]:
accuracies = []

# load mnist data
train_X, train_y, test_X, test_y = load_mnist_flat(proportion=1.0)

# iterate over k candidates
for k in K_CANDIDATES:
    # fit and predict our custom knn classifier
    pred = knn_classifier_L1(k, train_X, train_y, test_X)
    
    # calculate accuracy
    acc = np.mean(pred == test_y) # the mean of pred T/F array
    accuracies.append(acc)
    print(f"K={k}, accuracy={acc:.8f}")

MNIST
train shapes: (60000, 784) (60000,) 
test shapes: (10000, 784) (10000,)
K=1, accuracy=0.96310000
K=2, accuracy=0.96310000


In [ ]:
# plot the results
plt.figure(figsize=(10, 5))
plt.plot(K_CANDIDATES, accuracies, marker="o")
plt.title("KNN L1 Classifier: Accuracy vs K")
plt.xlabel("K Neighbors")
plt.ylabel("Accuracy")
plt.grid()
plt.xticks(K_CANDIDATES)
plt.show()